In [1]:
from keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input, Convolution2D, MaxPool2D, Dense, Flatten, Dropout
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
from keras.utils import plot_model

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from pathlib import Path
import pandas as pd
import numpy as np
import cv2

from datetime import datetime
import os
import tensorflow as tf

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

# Image size
img_width, img_height, img_depth = 150, 150, 3

ModuleNotFoundError: No module named 'cv2'

In [ ]:
#Training parameters
epochs = 50
freq = 20
batch_size = 32
num_classes = 2
train_valid_split = 0.2

act_type = 'sigmoid'
class_mode = 'binary'
loss_fun = 'binary_crossentropy'

fold_num = 1
#TODO: Change path
path = 'split_train/train'

In [ ]:
ds_train = tf.keras.preprocessing.image_dataset_from_directory(
    path,
    labels='inferred',
    label_mode='binary',
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(img_width, img_height),
    shuffle=True,
    seed=123,
    validation_split=train_valid_split,
    subset='training'
)

ds_valid = tf.keras.preprocessing.image_dataset_from_directory(
    path,
    labels='inferred',
    label_mode='binary',
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(img_width, img_height),
    shuffle=True,
    seed=123,
    validation_split=train_valid_split,
    subset='validation'
)


In [ ]:
# Use tf.keras.layers.preprocessing for augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(0.5),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomTranslation(0.2, 0.2)
    # Rotates by up to 90 degrees (0.5 * 180)
])

# Prepare the datasets for training and validation
# Apply data augmentation only to the training dataset
ds_train = ds_train.map(lambda x, y: (data_augmentation(x, training=True), y))

# You might want to add a rescaling layer as the first layer in your model
# instead of in the data augmentation pipeline, as it's applied to all data.
# However, for demonstration, we can include it here.
rescale = tf.keras.layers.Rescaling(1./255)

ds_train = ds_train.map(lambda x, y: (rescale(x), y))
ds_valid = ds_valid.map(lambda x, y: (rescale(x), y))

# Optimize performance
AUTOTUNE = tf.data.AUTOTUNE

ds_train = ds_train.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
ds_valid = ds_valid.cache().prefetch(buffer_size=AUTOTUNE)


In [ ]:
#InceptionV3 Model
image_input = Input(shape = (img_width, img_height, img_depth))
res_model = applications.ResNet152V2(input_tensor = image_input,
                                 include_top = False,
                                 weights = 'imagenet')
res_output = res_model.layers[-1].output

flat1 = Flatten()(res_output)
fc1 = Dense(512, activation = 'relu')(flat1)
dropfc1 = Dropout(0.5)(fc1)
fc2 = Dense(256, activation = 'relu')(dropfc1)
dropfc2 = Dropout(0.5)(fc2)

output_res = Dense(1, activation = act_type)(dropfc2)

for layer in res_model.layers[:-5]:
    layer.trainable = False

model_res = Model(image_input, output_res)

In [ ]:
# Compile the model
opt = Adam(learning_rate=0.00001)
model_res.compile(loss = loss_fun, optimizer = opt, metrics = ['accuracy'])

In [ ]:
# Folder setup
fold_num = 1

init_time = datetime.now()
current_time = init_time.strftime('%Y%m%d_%H%M%S')
name_dir = 'res_trained_models_' + current_time + '_fold_num' + str(fold_num)
os.mkdir(name_dir)

tensorboard = TensorBoard(
    log_dir='./logs',          # Directory for logs
    histogram_freq=1,          # How often to compute histograms
    write_graph=True,          # Write computation graph
    write_images=True          # Write model weights as images
)

# Callbacks3: EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='auto')

In [ ]:
history_res = model_res.fit(ds_train,
                              epochs = epochs,
                              steps_per_epoch = 16000 // batch_size,
                              validation_data = ds_valid,
                              validation_steps = 4000 // batch_size,
                              verbose=1,
                              callbacks = [early_stop, tensorboard])

early_stop_name = name_dir + '/fold_num_' + str(fold_num) + 'res_early_stop_model.weights.h5'
model_res.save_weights(early_stop_name)